In [30]:
pip install numpy pandas matplotlib scikit-learn tensorflow


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam

In [14]:

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/stock/prices.csv.zip')

In [23]:
# Preprocess the data
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert to datetime, coerce errors
df = df.dropna(subset=['date'])  # Drop rows with NaT values
df.sort_values('date', inplace=True)

# Use only the closing prices
data = df[['close']].values


In [24]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [25]:
# Create the dataset for training
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60  # Number of previous days to consider for prediction
X, y = create_dataset(scaled_data, time_step)

# Reshape X to be [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

In [26]:
# Split the data into training and test sets
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [27]:
# Build the GRU model
model = Sequential()
model.add(GRU(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(GRU(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [28]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))

# Predict the test set
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

Epoch 1/20
46/46 [==============================] - 8s 87ms/step - loss: 0.0018 - val_loss: 0.1274
Epoch 2/20
46/46 [==============================] - 5s 98ms/step - loss: 6.3756e-04 - val_loss: 0.1291
Epoch 3/20
46/46 [==============================] - 3s 62ms/step - loss: 6.3444e-04 - val_loss: 0.1326
Epoch 4/20
46/46 [==============================] - 3s 61ms/step - loss: 6.1409e-04 - val_loss: 0.1386
Epoch 5/20
46/46 [==============================] - 3s 62ms/step - loss: 5.5884e-04 - val_loss: 0.1456
Epoch 6/20
46/46 [==============================] - 5s 102ms/step - loss: 5.7185e-04 - val_loss: 0.1481
Epoch 7/20
46/46 [==============================] - 4s 77ms/step - loss: 6.1731e-04 - val_loss: 0.1607
Epoch 8/20
46/46 [==============================] - 3s 62ms/step - loss: 5.0639e-04 - val_loss: 0.1741
Epoch 9/20
46/46 [==============================] - 3s 62ms/step - loss: 4.9639e-04 - val_loss: 0.1860
Epoch 10/20
46/46 [==============================] - 3s 64ms/step - loss: 5.

In [29]:
# Inverse transform the predictions
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))